In [1]:
import classla
from pathlib import Path
from src.nlp.model import EntType
from src.nlp.text_processor import TextProcessor
from src.parse import (
    _find_entity,
    _map_titles_to_abbreviations,
    _normalize_input_for_clen,
    pdf_to_text,
)
import re
from spacy import displacy

classla.download("sl")

2025-02-28 08:05:57 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2025-02-28 08:05:58 INFO: File exists: /Users/tadejkrivec/classla_resources/sl/pos/standard.pt.
2025-02-28 08:05:58 INFO: File exists: /Users/tadejkrivec/classla_resources/sl/lemma/standard.pt.
2025-02-28 08:05:58 INFO: File exists: /Users/tadejkrivec/classla_resources/sl/depparse/standard.pt.
2025-02-28 08:05:58 INFO: File exists: /Users/tadejkrivec/classla_resources/sl/ner/standard.pt.
2025-02-28 08:05:58 INFO: File exists: /Users/tadejkrivec/classla_resources/sl/pretrain/standard.pt.
2025-02-28 08:05:58 INFO: Finished downloading models and saved to /Users/tadejkrivec/classla_resources.


In [2]:
tp = TextProcessor(mappings_path=Path("./data/mappings/mappings_all.parquet"))

2025-02-28 08:05:58 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |

2025-02-28 08:05:58 INFO: Use device: cpu
2025-02-28 08:05:58 INFO: Loading: tokenize
2025-02-28 08:05:58 INFO: Loading: pos
2025-02-28 08:06:01 INFO: Loading: lemma
2025-02-28 08:06:04 INFO: Done loading processors!


In [3]:
# https://www.sodnapraksa.si/?q=Ips%2023638/2021&database[SOVS]=SOVS&_submit=i%C5%A1%C4%8Di&rowsPerPage=20&page=0&id=2015081111479996
# file_path = "./data/test/I Ips 23638-2021.pdf"

# https://www.sodnapraksa.si/?q=II%20Ips%2032/2024&database[SOVS]=SOVS&_submit=i%C5%A1%C4%8Di&rowsPerPage=20&page=0&id=2015081111477717
file_path = "./data/test/II Ips 32-2024.pdf"

# https://www.sodnapraksa.si/?q=III%20Ips%2011/2024&database[SOVS]=SOVS&_submit=i%C5%A1%C4%8Di&rowsPerPage=20&page=0&id=2015081111480014
# file_path = "./data/test/III Ips 11-2024.pdf"

text_to_process = pdf_to_text(file_path=file_path)
text_to_process = text_to_process.replace("\n", " ")

In [4]:
test = tp.nlp(text_to_process)

In [5]:
entities_types_to_parse = [
    EntType.DOC_TITLE,
    EntType.DOC_ABBR,
    EntType.CLEN,
    EntType.CLEN_LEFT,
]

type_to_found_entities = {x: _find_entity(test, x) for x in entities_types_to_parse}

In [6]:
test.spans["sc"] = test.spans["spans"]

colors = {
    "NAVEDBA_ZAKONA": "orange",
    "STAVEK_Z_ZAKONOM": "lightblue",
    "POMEMBEN_IZSEK_BESEDILA": "red",
}
options = {
    "spans": ["NAVEDBA_ZAKONA", "STAVEK_Z_ZAKONOM", "POMEMBEN_IZSEK_BESEDILA"],
    "colors": colors,
}
displacy.render(test, style="span", options=options)

In [7]:
colors = {"DOC_TITLE": "red", "DOC_ABBR": "orange", "CLEN": "lightblue"}
options = {"ents": ["DOC_TITLE", "DOC_ABBR", "CLEN"], "colors": colors}
displacy.render(test, style="ent", options=options)

In [8]:
# zveza_found = []

# navedbe_zakona = [x for x in test.spans["spans"] if x.label_ == "navedba_zakona"]
# for span in navedbe_zakona:
#     entities_types_to_parse = [
#         EntType.DOC_TITLE,
#         EntType.DOC_ABBR,
#         EntType.CLEN,
#         EntType.CLEN_LEFT,
#     ]

#     # Extract abbreviations
#     type_to_found_entities = {x: _find_entity(span, x) for x in entities_types_to_parse}
#     _abbr = _map_titles_to_abbreviations(
#         titles=type_to_found_entities[EntType.DOC_TITLE], tp=tp
#     )
#     _abbr.extend(type_to_found_entities[EntType.DOC_ABBR])
#     found_abbr = max([str(x) for x in _abbr], key=len)

#     # Extract clens
#     nums = []
#     for x in span.text.split(" "):
#         pattern = r"\d{1,4}\."
#         matches = []
#         for word in x:
#             if re.match(pattern, word):
#                 matches.append(word)

#     if len(nums) > 1:
#         for num in nums:
#             zveza_found.append(f"{num} člen {found_abbr}")
#     else:
#         clens = [
#             *[str(x) for x in type_to_found_entities[EntType.CLEN]],
#             *[str(x) for x in type_to_found_entities[EntType.CLEN_LEFT]],
#         ]

#         if len(clens) != 1:
#             print(clens)
#             raise ValueError

#         for x in clens:
#             found_clen = _normalize_input_for_clen(str(x))
#             zveza_found.append(f"{found_clen} {found_abbr}")

#         if not found_clen.split(" ")[0].strip(".").isdigit():
#             found_num = found_clen.split(".")[0]
#             zveza_found.append(f"{found_num}. člen {found_abbr}")

# zveza_found = sorted(list(set(zveza_found)), key=lambda x: int(x.split(".")[0]))
